# 1. Set up

In [1]:
!pip install neo4j
!pip install torch
!pip install torch_geometric

# 2. Import necessary libraries

In [2]:
from neo4j import GraphDatabase

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.data import Data, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from typing import Dict, List
import warnings
from config import *
warnings.filterwarnings('ignore')

# 3. Define global variables

In [4]:
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "password"

# 4. Functions

In [5]:
def run_query(query, parameters=None):
    """
    Function used to run a Cypher query against a neo4j graph database.

    Parameters:
        - query: The query itself.
        - parameters: 
    """
    with driver.session() as session:
        result = session.run(query, parameters)
        return [record for record in result]

# 5. Code

Vamos a parar la parte de la extracción y pegamos abajo el código que nos proporciona claude para crear el esqueleto del modelado de la red neuronal de grafos

**Librerías que necesitamos -> torch, torch_geometric, sklearn, numpy, pandas, seaborn**

## 5.2. Data processing

We perform the data processing:

In [6]:
class FraudGraphDataProcessor:
    """
    Procesador de datos para crear grafos de detección de fraudes
    """
    
    def __init__(self):
        self.scaler = StandardScaler() # Para normalizar características numéricas para evitar que una característica domine sobre otras.
        self.label_encoder = LabelEncoder() # Para codificar etiquetas
        self.node_features = None # Guardará las características procesadas
        self.edge_index = None # Guardará las conexiones del grafo
        self.labels = None # Guardará las etiquetas de clase
        
    def load_neo4j_data(self, neo4j_connection) -> Dict:
        """
        Carga datos desde Neo4j (adaptable a tu conexión)
        """
        # Consulta para obtener nodos CONTADOR con sus características (marca, modelo, potencia, ...)
        contador_query = """
        MATCH (c:CONTADOR)
        OPTIONAL MATCH (c)-[:INVOLUCRADO_EN_FRAUDE]->(ef:EXPEDIENTE_FRAUDE)
        OPTIONAL MATCH (c)-[:MIDE_CONSUMO_DE]->(s:SUMINISTRO)
        OPTIONAL MATCH (c)-[:INSTALADO_EN]->(u:UBICACION)
        OPTIONAL MATCH (c)-[:GENERA_MEDICION]->(m:MEDICION)
        RETURN 
            c.nis_rad as node_id,
            c.marca_contador as marca,
            c.modelo_contador as modelo,
            c.estado_tg as estado_tg,
            c.telegest_activo as telegest_activo,
            c.potencia_maxima as potencia_maxima,
            c.fases_contador as fases,
            s.potencia_contratada as potencia_contratada,
            s.estado_contrato as estado_contrato,
            u.coordenada_x as coord_x,
            u.coordenada_y as coord_y,
            avg(m.energia_activa) as consumo_promedio,
            stdDev(m.energia_activa) as consumo_variabilidad,
            count(m) as num_mediciones,
            ef.tipo_anomalia as label,
            ef.clasificacion_fraude as tipo_fraude,
            ef.valoracion_total as impacto_economico
        """
        
        # Consulta para obtener relaciones:
        # - proximidad geográfica (contadores a menos de 5km)
        # - Misma comercializadora (misma empresa suministradora)
        # - Mismo modelo (características técnicas similares)
        # - Mismo concentrador (Infraestructura compartida)
        edges_query = """
        MATCH (c1:CONTADOR)-[:INSTALADO_EN]->(u:UBICACION)<-[:INSTALADO_EN]-(c2:CONTADOR)
        WHERE c1 <> c2
        AND point.distance(
            point({x: u.coordenada_x, y: u.coordenada_y}),
            point({x: u.coordenada_x, y: u.coordenada_y})
        ) < 5000
        RETURN c1.nis_rad as source, c2.nis_rad as target, 'CERCANO' as relation_type
        
        UNION
        
        MATCH (c1:CONTADOR)-[:MIDE_CONSUMO_DE]->(s:SUMINISTRO)-[:CONTRATADO_CON]->(com:COMERCIALIZADORA)
              <-[:CONTRATADO_CON]-(s2:SUMINISTRO)<-[:MIDE_CONSUMO_DE]-(c2:CONTADOR)
        WHERE c1 <> c2
        RETURN c1.nis_rad as source, c2.nis_rad as target, 'MISMA_COMERCIALIZADORA' as relation_type
        """
        
        # Simulación de datos. Más adelante lo modificaremos con nuestra conexión real
        return self._simulate_data()
    
    def _simulate_data(self) -> Dict:
        """
        Simula datos basados en el esquema de Neo4j. Creamos datos sintéticos para testing y desarrollo cuando no 
        tenemos acceso a neo4j
        """
        np.random.seed(RANDOM_SEED)
        
        # Crear nodos de contadores, en este caso crearemos 10500:
        n_nodes = 10000
        n_fraud = 500  # Setearemos un 5% fraudes
        
        # Características de nodos
        node_data = {
            'node_id': [f"NIS_{i:06d}" for i in range(n_nodes)],
            'marca': np.random.choice(['MARCA_A', 'MARCA_B', 'MARCA_C'], n_nodes),
            'modelo': np.random.choice(['MOD_1', 'MOD_2', 'MOD_3'], n_nodes),
            'estado_tg': np.random.choice(['INTEGRADO', 'NO_INTEGRADO'], n_nodes, p=[0.8, 0.2]),
            'telegest_activo': np.random.choice([True, False], n_nodes, p=[0.9, 0.1]),
            'potencia_maxima': np.random.normal(5000, 2000, n_nodes),
            'potencia_contratada': np.random.normal(3000, 1500, n_nodes),
            'coord_x': np.random.uniform(400000, 600000, n_nodes),
            'coord_y': np.random.uniform(4400000, 4800000, n_nodes),
            'consumo_promedio': np.random.exponential(50, n_nodes),
            'consumo_variabilidad': np.random.exponential(20, n_nodes),
            'num_mediciones': np.random.poisson(100, n_nodes),
        }
        
        # Crear etiquetas (la mayoría normal, algunos fraudes)
        labels = ['NORMAL'] * n_nodes
        fraud_indices = np.random.choice(n_nodes, n_fraud, replace=False)
        
        for i in fraud_indices:
            labels[i] = np.random.choice(['FRAUDE', 'IRREGULARIDAD'], p=[0.7, 0.3])
            # Hacer que los fraudes tengan patrones diferentes
            if labels[i] == 'FRAUDE':
                node_data['consumo_promedio'][i] *= 0.3  # Consumo muy bajo
                node_data['consumo_variabilidad'][i] *= 0.5  # Menor variabilidad
            
        node_data['label'] = labels
        
        # Crear las relaciones entre nodos
        edges = []
        
        # Bordes por proximidad geográfica
        for i in range(n_nodes):
            for j in range(i+1, min(i+20, n_nodes)):  # Limitar para eficiencia
                dist = np.sqrt((node_data['coord_x'][i] - node_data['coord_x'][j])**2 + 
                              (node_data['coord_y'][i] - node_data['coord_y'][j])**2)
                if dist < 5000:  # 5km
                    edges.append([i, j])
                    edges.append([j, i])  # Borde bidireccional
        
        # Bordes por similaridad técnica
        for i in range(n_nodes):
            for j in range(i+1, n_nodes):
                if (node_data['marca'][i] == node_data['marca'][j] and 
                    node_data['modelo'][i] == node_data['modelo'][j] and
                    np.random.random() < 0.1):  # 10% de probabilidad
                    edges.append([i, j])
                    edges.append([j, i])
        
        return {
            'nodes': pd.DataFrame(node_data),
            'edges': np.array(edges) if edges else np.array([]).reshape(0, 2)
        }
    
    def create_graph_data(self, data: Dict) -> Data:
        """
        Convierte datos en formato Data de PyTorch Geometric.
        """
        
        nodes_df = data['nodes']
        edges = data['edges']
        
        # Mapear node_ids a índices
        node_to_idx = {node_id: idx for idx, node_id in enumerate(nodes_df['node_id'])}
        
        # Preparar características de nodos
        feature_columns = [
            'potencia_maxima', 'potencia_contratada', 'coord_x', 'coord_y',
            'consumo_promedio', 'consumo_variabilidad', 'num_mediciones'
        ]
        
        # Codificar características categóricas con one hot encoding en este caso:
        marca_encoded = pd.get_dummies(nodes_df['marca'], prefix='marca')
        modelo_encoded = pd.get_dummies(nodes_df['modelo'], prefix='modelo')
        estado_tg_encoded = pd.get_dummies(nodes_df['estado_tg'], prefix='estado_tg')
        
        # Características binarias
        telegest_feature = nodes_df['telegest_activo'].astype(int)
        
        # Combinar todas las características
        numeric_features = self.scaler.fit_transform(nodes_df[feature_columns])

        # combinamos todas las características:
        node_features = np.concatenate([
            numeric_features,
            marca_encoded.values,
            modelo_encoded.values,
            estado_tg_encoded.values,
            telegest_feature.values.reshape(-1, 1)
        ], axis=1)
        
        # Preparar etiquetas
        labels = [FRAUD_CLASSES[label] for label in nodes_df['label']]
        
        # Crear tensores
        x = torch.tensor(node_features, dtype=torch.float) # características
        y = torch.tensor(labels, dtype=torch.long) # etiquetas
        
        # Crear edge_index
        if len(edges) > 0:
            # primera fila nodos origen y segunda fila nodo destino
            edge_index = torch.tensor(edges.T, dtype=torch.long)
        else:
            edge_index = torch.empty((2, 0), dtype=torch.long)
        
        # Crear objeto Data final:
        graph_data = Data(x=x, edge_index=edge_index, y=y)
        
        # Guardar información para uso posterior
        self.node_features = node_features
         self.labels = labels
        self.feature_names = (feature_columns + 
                             list(marca_encoded.columns) + 
                             list(modelo_encoded.columns) + 
                             list(estado_tg_encoded.columns) + 
                             ['telegest_activo'])
        
        return graph_data

IndentationError: unexpected indent (3096986611.py, line 191)

## 5.3. Modelo GNN

Vamos a construir la arquitectura, el esqueleto de la red neuronal GNN que será o bien una GCN o bien una GAT

In [ ]:
from arquitectura_GNN_homo import FraudGNN

## 5.4. Entrenamiento

Construcción de la clase utilizada para entrenar el modelo GNN definido en 5.3.

In [ ]:
from trainer import FraudTrainer

## 5.5. Evaluación

In [ ]:
from evaluator import FraudEvaluator

## 5.6. Construcción del Pipeline

In [ ]:
# ==========================================
# 6. PIPELINE PRINCIPAL
# ==========================================

def main():
    """Pipeline principal de entrenamiento"""
    
    # Configurar dispositivo
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando dispositivo: {device}")
    
    # 1. Cargar y procesar datos
    print("1. Cargando datos...")
    processor = FraudGraphDataProcessor()
    raw_data = processor.load_neo4j_data(None)  # Usar datos simulados
    graph_data = processor.create_graph_data(raw_data)
    
    print(f"   - Nodos: {graph_data.num_nodes}")
    print(f"   - Bordes: {graph_data.num_edges}")
    print(f"   - Características: {graph_data.num_node_features}")
    
    # 2. Dividir datos
    print("2. Dividiendo datos...")
    num_nodes = graph_data.num_nodes
    # Use randperm in order to reorder the indexes
    indices = torch.randperm(num_nodes)
    
    train_size = int(0.6 * num_nodes)
    val_size = int(0.2 * num_nodes)
    
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Just to select train, test and val data:
    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True
    
    # 3. Crear modelo
    print("3. Creando modelo...")
    model = FraudGNN(
        input_dim=graph_data.num_node_features,
        hidden_dim=HIDDEN_DIM,
        num_classes=len(FRAUD_CLASSES),
        model_type='GCN'  # o 'GAT'
    )
    # p.numel devuelve el número de elementos que tiene un tensor.
    print(f"   - Parámetros del modelo: {sum(p.numel() for p in model.parameters())}")
    
    # 4. Entrenar
    print("4. Entrenando modelo...")
    trainer = FraudTrainer(model, device)
    graph_data = graph_data.to(device)
    train_mask = train_mask.to(device)
    val_mask = val_mask.to(device)
    test_mask = test_mask.to(device)
    
    history = trainer.train(graph_data, train_mask, val_mask)
    
    # 5. Evaluar
    print("5. Evaluando modelo...")
    model.load_state_dict(torch.load('best_fraud_model.pth'))
    evaluator = FraudEvaluator(model, device)
    
    test_results = evaluator.evaluate_detailed(graph_data, test_mask)
    
    # 6. Mostrar resultados
    print("6. Resultados:")
    print(f"   - Mejor precisión de validación: {history['best_val_acc']:.4f}")
    print(f"   - ROC AUC: {test_results['roc_auc']:.4f}")
    print("\nReporte de clasificación:")
    print(test_results['classification_report'])
    
    # 7. Visualizar
    print("7. Generando visualizaciones...")
    evaluator.plot_training_history(history)
    evaluator.plot_confusion_matrix(test_results['confusion_matrix'])
    
    # 8. Detectar fraudes potenciales
    print("8. Detectando fraudes potenciales...")
    all_predictions = evaluator.predict(graph_data)
    all_probabilities = evaluator.predict_proba(graph_data)
    
    # Casos con alta probabilidad de fraude pero sin etiqueta
    unlabeled_fraud_candidates = []
    for i in range(len(all_predictions)):
        if graph_data.y[i] == 0:  # Etiquetado como normal
            fraud_prob = all_probabilities[i][1] + all_probabilities[i][2]
            if fraud_prob > 0.7:  # Alta probabilidad de fraude
                unlabeled_fraud_candidates.append({
                    'node_index': i,
                    'fraud_probability': fraud_prob,
                    'predicted_class': all_predictions[i]
                })
    
    print(f"   - Casos sospechosos encontrados: {len(unlabeled_fraud_candidates)}")
    
    return {
        'model': model,
        'history': history,
        'test_results': test_results,
        'fraud_candidates': unlabeled_fraud_candidates,
        'processor': processor
    }

if __name__ == "__main__":
    # Ejecutar pipeline
    torch.manual_seed(RANDOM_SEED)
    np.random.seed(RANDOM_SEED)
    
    results = main()
    
    print("\n" + "="*50)
    print("ENTRENAMIENTO COMPLETADO")
    print("="*50)
    print(f"Modelo guardado en: best_fraud_model.pth")
    print(f"Fraudes potenciales detectados: {len(results['fraud_candidates'])}")

## 5.7. Integración con Neo4j real

In [ ]:
# ==========================================
# 7. INTEGRACIÓN CON NEO4J REAL
# ==========================================

class Neo4jGraphLoader:
    """Cargador específico para datos reales de Neo4j"""
    
    def __init__(self, uri: str, user: str, password: str):
        try:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
            # Test de conexión
            with self.driver.session() as session:
                result = session.run("RETURN 1 as test")
                test_val = result.single()["test"]
                if test_val == 1:
                    print("✅ Conexión a Neo4j establecida correctamente")
                else:
                    raise Exception("Error en test de conexión")
        except Exception as e:
            raise Exception(f"❌ Error conectando a Neo4j: {e}")
        
    def close(self):
        if self.driver:
            self.driver.close()
            print("🔌 Conexión a Neo4j cerrada")
        
    def load_fraud_graph_data(self) -> Dict:
        """Carga datos reales desde tu base de datos Neo4j"""
        
        with self.driver.session() as session:
            # Consulta optimizada para obtener todas las características de nodos
            nodes_query = """
            MATCH (c:CONTADOR)
            OPTIONAL MATCH (c)-[:INVOLUCRADO_EN_FRAUDE]->(ef:EXPEDIENTE_FRAUDE)
            OPTIONAL MATCH (c)-[:MIDE_CONSUMO_DE]->(s:SUMINISTRO)
            OPTIONAL MATCH (c)-[:INSTALADO_EN]->(u:UBICACION)
            OPTIONAL MATCH (c)-[:CONECTADO_A]->(conc:CONCENTRADOR)
            OPTIONAL MATCH (c)-[:GENERA_MEDICION]->(m:MEDICION)
            OPTIONAL MATCH (c)-[:GENERA_EVENTO]->(e:EVENTO)
            OPTIONAL MATCH (c)-[:INSPECCIONADO_EN]->(i:INSPECCION)
            
            WITH c, ef, s, u, conc,
                 count(DISTINCT m) as num_mediciones,
                 avg(m.energia_activa) as consumo_promedio,
                 stdDev(m.energia_activa) as consumo_variabilidad,
                 min(m.timestamp_medicion) as primera_medicion,
                 max(m.timestamp_medicion) as ultima_medicion,
                 count(DISTINCT e) as num_eventos,
                 count(DISTINCT i) as num_inspecciones,
                 collect(DISTINCT e.tipo_reporte) as tipos_eventos
                 
            RETURN 
                c.nis_rad as node_id,
                c.numero_contador as numero_contador,
                c.marca_contador as marca,
                c.modelo_contador as modelo,
                c.tipo_aparato as tipo_aparato,
                c.estado_tg as estado_tg,
                c.telegest_activo as telegest_activo,
                c.potencia_maxima as potencia_maxima,
                c.fases_contador as fases,
                c.tension as tension,
                c.version_firmware as version_firmware,
                
                s.estado_contrato as estado_contrato,
                s.potencia_contratada as potencia_contratada,
                s.tension_suministro as tension_suministro,
                s.tipo_punto as tipo_punto,
                s.cnae as cnae,
                s.tarifa_activa as tarifa,
                s.comercializadora_codigo as comercializadora,
                
                u.coordenada_x as coord_x,
                u.coordenada_y as coord_y,
                u.codigo_postal as codigo_postal,
                u.area_ejecucion as area_ejecucion,
                
                conc.estado_comunicacion as estado_comunicacion,
                conc.version_concentrador as version_concentrador,
                
                num_mediciones,
                consumo_promedio,
                consumo_variabilidad,
                primera_medicion,
                ultima_medicion,
                num_eventos,
                num_inspecciones,
                tipos_eventos,
                
                ef.tipo_anomalia as label,
                ef.clasificacion_fraude as tipo_fraude,
                ef.valoracion_total as impacto_economico,
                ef.energia_facturada as energia_facturada,
                ef.fecha_inicio_anomalia as fecha_inicio_fraude,
                ef.fecha_fin_anomalia as fecha_fin_fraude
            """
            
            # Consulta para relaciones del grafo
            edges_query = """
            // Relaciones por proximidad geográfica (< 2km)
            MATCH (c1:CONTADOR)-[:INSTALADO_EN]->(u1:UBICACION)
            MATCH (c2:CONTADOR)-[:INSTALADO_EN]->(u2:UBICACION)
            WHERE c1 <> c2
            AND point.distance(
                point({x: u1.coordenada_x, y: u1.coordenada_y}),
                point({x: u2.coordenada_x, y: u2.coordenada_y})
            ) < 2000
            RETURN c1.nis_rad as source, c2.nis_rad as target, 
                   'PROXIMIDAD_GEOGRAFICA' as relation_type,
                   point.distance(
                       point({x: u1.coordenada_x, y: u1.coordenada_y}),
                       point({x: u2.coordenada_x, y: u2.coordenada_y})
                   ) as distancia
            
            UNION
            
            // Relaciones por misma comercializadora
            MATCH (c1:CONTADOR)-[:MIDE_CONSUMO_DE]->(s1:SUMINISTRO)
            MATCH (c2:CONTADOR)-[:MIDE_CONSUMO_DE]->(s2:SUMINISTRO)
            WHERE c1 <> c2 
            AND s1.comercializadora_codigo = s2.comercializadora_codigo
            RETURN c1.nis_rad as source, c2.nis_rad as target,
                   'MISMA_COMERCIALIZADORA' as relation_type,
                   s1.comercializadora_codigo as comercializadora
            
            UNION
            
            // Relaciones por misma marca y modelo
            MATCH (c1:CONTADOR), (c2:CONTADOR)
            WHERE c1 <> c2
            AND c1.marca_contador = c2.marca_contador
            AND c1.modelo_contador = c2.modelo_contador
            RETURN c1.nis_rad as source, c2.nis_rad as target,
                   'MISMO_MODELO' as relation_type,
                   c1.marca_contador + '_' + c1.modelo_contador as modelo
            
            UNION
            
            // Relaciones por mismo concentrador
            MATCH (c1:CONTADOR)-[:CONECTADO_A]->(conc:CONCENTRADOR)<-[:CONECTADO_A]-(c2:CONTADOR)
            WHERE c1 <> c2
            RETURN c1.nis_rad as source, c2.nis_rad as target,
                   'MISMO_CONCENTRADOR' as relation_type,
                   conc.concentrador_id as concentrador_id
            """
            
            # Ejecutar consultas
            nodes_result = session.run(nodes_query)
            edges_result = session.run(edges_query)
            
            # Convertir a DataFrames
            nodes_df = pd.DataFrame([dict(record) for record in nodes_result])
            edges_df = pd.DataFrame([dict(record) for record in edges_result])
            
            return {
                'nodes': nodes_df,
                'edges': edges_df
            }

## 5.8. Análisis temporal avanzado

In [ ]:
# ==========================================
# 8. ANÁLISIS TEMPORAL AVANZADO
# ==========================================

class TemporalFraudAnalyzer:
    """Análisis temporal para detección de fraudes"""
    
    def __init__(self, neo4j_loader: Neo4jGraphLoader):
        self.loader = neo4j_loader
        
    def extract_temporal_features(self, nis_rad: str, days_back: int = 365) -> Dict:
        """Extrae características temporales específicas para un contador"""
        
        with self.loader.driver.session() as session:
            query = """
            MATCH (c:CONTADOR {nis_rad: $nis_rad})-[:GENERA_MEDICION]->(m:MEDICION)
            WHERE m.timestamp_medicion >= datetime() - duration('P' + $days_back + 'D')
            WITH c, m
            ORDER BY m.timestamp_medicion
            
            WITH c, collect(m) as mediciones
            
            // Calcular características temporales
            WITH c, mediciones,
                 [m IN mediciones | m.energia_activa] as consumos,
                 [i IN range(1, size(mediciones)-1) | 
                    mediciones[i].energia_activa - mediciones[i-1].energia_activa] as diferencias
            
            RETURN 
                // Estadísticas básicas
                avg([m IN mediciones | m.energia_activa]) as consumo_promedio,
                stdDev([m IN mediciones | m.energia_activa]) as consumo_std,
                min([m IN mediciones | m.energia_activa]) as consumo_min,
                max([m IN mediciones | m.energia_activa]) as consumo_max,
                
                // Detección de cambios bruscos
                size([d IN diferencias WHERE abs(d) > 100]) as cambios_bruscos,
                max([d IN diferencias | abs(d)]) as max_cambio,
                
                // Patrones de resistencia (indicador de manipulación)
                avg([m IN mediciones | m.resistencia_r1]) as resistencia_r1_promedio,
                stdDev([m IN mediciones | m.resistencia_r1]) as resistencia_r1_std,
                
                // Consistencia temporal
                size(mediciones) as total_mediciones,
                duration.between(mediciones[0].timestamp_medicion, 
                               mediciones[-1].timestamp_medicion).days as periodo_dias
            """
            
            result = session.run(query, nis_rad=nis_rad, days_back=days_back)
            record = result.single()
            
            if record:
                return dict(record)
            return {}
    
    def detect_consumption_anomalies(self, threshold_std: float = 3.0) -> List[Dict]:
        """Detecta anomalías en patrones de consumo"""
        
        with self.loader.driver.session() as session:
            query = """
            MATCH (c:CONTADOR)-[:GENERA_MEDICION]->(m:MEDICION)
            WHERE m.timestamp_medicion >= datetime() - duration('P90D')
            
            WITH c, 
                 avg(m.energia_activa) as consumo_promedio,
                 stdDev(m.energia_activa) as consumo_std,
                 collect(m.energia_activa) as consumos,
                 count(m) as num_mediciones
            
            WHERE consumo_std > 0 AND num_mediciones > 10
            
            // Identificar mediciones anómalas
            WITH c, consumo_promedio, consumo_std, consumos,
                 [consumo IN consumos WHERE 
                    abs(consumo - consumo_promedio) > $threshold_std * consumo_std] as anomalias
            
            WHERE size(anomalias) > 0
            
            RETURN c.nis_rad as nis_rad,
                   consumo_promedio,
                   consumo_std,
                   size(anomalias) as num_anomalias,
                   size(consumos) as total_mediciones,
                   (toFloat(size(anomalias)) / size(consumos)) as porcentaje_anomalias
            
            ORDER BY porcentaje_anomalias DESC
            """
            
            result = session.run(query, threshold_std=threshold_std)
            return [dict(record) for record in result]

# ==========================================
# 9. MODELO GNN HÍBRIDO CON SERIES TEMPORALES
# ==========================================

class HybridTemporalGNN(nn.Module):
    """GNN híbrido que combina características estáticas y temporales"""
    
    def __init__(self, static_dim: int, temporal_dim: int, hidden_dim: int = 64, 
                 num_classes: int = 3, sequence_length: int = 30):
        super(HybridTemporalGNN, self).__init__()
        
        # Encoder para características estáticas (GNN)
        self.static_conv1 = GCNConv(static_dim, hidden_dim)
        self.static_conv2 = GCNConv(hidden_dim, hidden_dim)
        
        # Encoder para series temporales (LSTM/GRU)
        self.temporal_encoder = nn.LSTM(
            temporal_dim, hidden_dim, batch_first=True, num_layers=2, dropout=0.2
        )
        
        # Capa de atención para combinar información
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=4, batch_first=True)
        
        # Clasificador final
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, static_x, temporal_x, edge_index, batch=None):
        # Procesar características estáticas con GNN
        static_out = F.relu(self.static_conv1(static_x, edge_index))
        static_out = F.dropout(static_out, training=self.training)
        static_out = self.static_conv2(static_out, edge_index)
        
        # Procesar series temporales con LSTM
        temporal_out, _ = self.temporal_encoder(temporal_x)
        temporal_out = temporal_out[:, -1, :]  # Tomar último estado
        
        # Combinar con atención
        combined_features = torch.cat([
            static_out.unsqueeze(1), 
            temporal_out.unsqueeze(1)
        ], dim=1)
        
        attended_features, _ = self.attention(
            combined_features, combined_features, combined_features
        )
        
        # Concatenar características finales
        final_features = torch.cat([
            attended_features[:, 0, :],  # Estáticas
            attended_features[:, 1, :]   # Temporales
        ], dim=1)
        
        # Clasificar
        output = self.classifier(final_features)
        return F.log_softmax(output, dim=1)

## 5.9. Monitoreo en tiempo real

In [ ]:
# ==========================================
# 10. SISTEMA DE MONITOREO EN TIEMPO REAL
# ==========================================

class RealTimeFraudMonitor:
    """Monitor en tiempo real para detección de fraudes"""
    
    def __init__(self, model_path: str, neo4j_loader: Neo4jGraphLoader,
                 threshold_fraud: float = 0.7):
        self.model = torch.load(model_path)
        self.model.eval()
        self.neo4j_loader = neo4j_loader
        self.threshold_fraud = threshold_fraud
        self.alert_history = []
        
    def check_new_measurements(self, hours_back: int = 24) -> List[Dict]:
        """Revisa mediciones recientes en busca de patrones fraudulentos"""
        
        with self.neo4j_loader.driver.session() as session:
            query = """
            MATCH (c:CONTADOR)-[:GENERA_MEDICION]->(m:MEDICION)
            WHERE m.timestamp_medicion >= datetime() - duration('PT' + $hours_back + 'H')
            
            // Agrupar por contador y calcular cambios
            WITH c, collect(m ORDER BY m.timestamp_medicion) as mediciones
            WHERE size(mediciones) >= 2
            
            WITH c, mediciones,
                 mediciones[-1].energia_activa as ultimo_consumo,
                 mediciones[-2].energia_activa as penultimo_consumo,
                 avg([m IN mediciones | m.energia_activa]) as consumo_promedio_reciente
            
            // Detectar caídas bruscas o patrones anómalos
            WHERE ultimo_consumo < (penultimo_consumo * 0.3)  // Caída >70%
               OR ultimo_consumo < (consumo_promedio_reciente * 0.2)  // Muy bajo vs promedio
            
            RETURN c.nis_rad as nis_rad,
                   ultimo_consumo,
                   penultimo_consumo,
                   consumo_promedio_reciente,
                   ((penultimo_consumo - ultimo_consumo) / penultimo_consumo) as porcentaje_caida
            
            ORDER BY porcentaje_caida DESC
            """
            
            result = session.run(query, hours_back=hours_back)
            suspicious_cases = [dict(record) for record in result]
            
            # Evaluar cada caso con el modelo GNN
            alerts = []
            for case in suspicious_cases:
                fraud_probability = self._evaluate_fraud_probability(case['nis_rad'])
                
                if fraud_probability > self.threshold_fraud:
                    alert = {
                        'timestamp': pd.Timestamp.now(),
                        'nis_rad': case['nis_rad'],
                        'fraud_probability': fraud_probability,
                        'trigger': 'CAIDA_BRUSCA_CONSUMO',
                        'details': case,
                        'priority': 'HIGH' if fraud_probability > 0.9 else 'MEDIUM'
                    }
                    alerts.append(alert)
                    self.alert_history.append(alert)
            
            return alerts
    
    def _evaluate_fraud_probability(self, nis_rad: str) -> float:
        """Evalúa probabilidad de fraude para un contador específico"""
        # Aquí implementarías la evaluación con el modelo GNN
        # Por simplicidad, retorna un valor simulado
        return np.random.random()
    
    def generate_daily_report(self) -> Dict:
        """Genera reporte diario de actividad fraudulenta"""
        
        recent_alerts = [alert for alert in self.alert_history 
                        if alert['timestamp'] > pd.Timestamp.now() - pd.Timedelta(days=1)]
        
        return {
            'fecha': pd.Timestamp.now().date(),
            'total_alertas': len(recent_alerts),
            'alertas_alta_prioridad': len([a for a in recent_alerts if a['priority'] == 'HIGH']),
            'alertas_media_prioridad': len([a for a in recent_alerts if a['priority'] == 'MEDIUM']),
            'tipos_triggers': pd.Series([a['trigger'] for a in recent_alerts]).value_counts().to_dict(),
            'contadores_afectados': list(set([a['nis_rad'] for a in recent_alerts])),
            'probabilidad_fraude_promedio': np.mean([a['fraud_probability'] for a in recent_alerts]) if recent_alerts else 0
        }

## 5.10. Uso completo

In [ ]:
# ==========================================
# 11. EJEMPLO DE USO COMPLETO
# ==========================================

def run_production_pipeline():
    """Pipeline completo para producción"""
    
    print("🚀 INICIANDO PIPELINE DE DETECCIÓN DE FRAUDES GNN")
    print("="*60)
    
    # 1. Conectar a Neo4j
    print("1. Conectando a Neo4j...")
    neo4j_loader = Neo4jGraphLoader(
        uri="bolt://localhost:7687",
        user="neo4j", 
        password="your_password"
    )
    
    try:
        # 2. Cargar datos reales
        print("2. Cargando datos desde Neo4j...")
        raw_data = neo4j_loader.load_fraud_graph_data()
        print(f"   ✅ Cargados {len(raw_data['nodes'])} nodos y {len(raw_data['edges'])} relaciones")
        
        # 3. Procesar datos para GNN
        print("3. Procesando datos para GNN...")
        processor = FraudGraphDataProcessor()
        graph_data = processor.create_graph_data(raw_data)
        print(f"   ✅ Grafo creado: {graph_data.num_nodes} nodos, {graph_data.num_edges} bordes")
        
        # 4. Entrenar modelo (si no existe)
        model_path = 'production_fraud_model.pth'
        if not os.path.exists(model_path):
            print("4. Entrenando modelo GNN...")
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            
            model = FraudGNN(
                input_dim=graph_data.num_node_features,
                hidden_dim=128,
                num_classes=3,
                model_type='GAT'  # Usar GAT para mejor rendimiento
            )
            
            trainer = FraudTrainer(model, device)
            
            # División de datos
            num_nodes = graph_data.num_nodes
            indices = torch.randperm(num_nodes)
            train_size = int(0.7 * num_nodes)
            val_size = int(0.15 * num_nodes)
            
            train_mask = torch.zeros(num_nodes, dtype=torch.bool)
            val_mask = torch.zeros(num_nodes, dtype=torch.bool)
            test_mask = torch.zeros(num_nodes, dtype=torch.bool)
            
            train_mask[indices[:train_size]] = True
            val_mask[indices[train_size:train_size + val_size]] = True
            test_mask[indices[train_size + val_size:]] = True
            
            # Entrenar
            graph_data = graph_data.to(device)
            history = trainer.train(graph_data, train_mask, val_mask)
            
            # Guardar modelo
            torch.save(model.state_dict(), model_path)
            print(f"   ✅ Modelo entrenado y guardado en {model_path}")
        
        # 5. Configurar monitoreo en tiempo real
        print("5. Configurando monitoreo en tiempo real...")
        monitor = RealTimeFraudMonitor(model_path, neo4j_loader)
        
        # 6. Ejecutar detección inicial
        print("6. Ejecutando detección de fraudes...")
        alerts = monitor.check_new_measurements(hours_back=48)
        print(f"   ⚠️  {len(alerts)} alertas de fraude detectadas")
        
        for alert in alerts[:5]:  # Mostrar primeras 5
            print(f"   🚨 {alert['nis_rad']}: {alert['fraud_probability']:.3f} "
                  f"({alert['priority']}) - {alert['trigger']}")
        
        # 7. Generar reporte
        print("7. Generando reporte diario...")
        daily_report = monitor.generate_daily_report()
        print(f"   📊 Total alertas: {daily_report['total_alertas']}")
        print(f"   🔴 Alta prioridad: {daily_report['alertas_alta_prioridad']}")
        print(f"   🟡 Media prioridad: {daily_report['alertas_media_prioridad']}")
        
        print("\n✅ PIPELINE COMPLETADO EXITOSAMENTE")
        return {
            'alerts': alerts,
            'report': daily_report,
            'model_path': model_path,
            'monitor': monitor
        }
        
    finally:
        neo4j_loader.close()

# Para ejecutar en producción:
# results = run_production_pipeline()